# --------------------------------------------------------------
# Setup the Environments      
# --------------------------------------------------------------

In [8]:
import os
import openai
from dotenv import load_dotenv
from rich import print
from pydantic import BaseModel, Field
from typing import Literal,Optional

load_dotenv()
API_HOST = os.getenv("API_HOST", "github")

if API_HOST == "github":
    print("Using GitHub models...")
    client = openai.OpenAI(base_url="https://models.github.ai/inference", api_key=os.environ["GITHUB_TOKEN"])
    MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")
elif API_HOST == "ollama":
    print("Using Ollama model on local...")
    client = openai.OpenAI(base_url="http://localhost:11434/v1", api_key="nokeyneeded")
    MODEL_NAME = os.getenv("OLLAMA_MODEL", "mistral")


Using GitHub models...

# --------------------------------------------------------------
# Step 1: Define the data models for routing and responses
# --------------------------------------------------------------

In [20]:
class CalendarRequestType(BaseModel):
    request_type: Literal["new_event", "modify_event", "other"]
    confidence_score:float
    description: str


class NewEventDetails(BaseModel):
    name: str = Field(description="Name of the event")
    date: str = Field(description="Date and time of the event (ISO 8601)")
    duration_minutes: int = Field(description="Duration in minutes")
    participants: list[str] = Field(description="List of participants")


class CalendarResponse(BaseModel):
    """Final response format"""
    success: bool = Field(description="Whether the operation was successful")
    message: str = Field(description="User-friendly response message")
    calendar_link: Optional[str] = Field(description="Calendar link if applicable")


class Change(BaseModel):
    """Details for changing an existing event"""

    field: str = Field(description="Field to change")
    new_value: str = Field(description="New value for the field")


class ModifyEventDetails(BaseModel):
    """Details for modifying an existing event"""
    event_identifier: str = Field(description="Description to identify the existing event")
    changes: list[Change] = Field(description="List of changes to make")
    participants_to_add: list[str] = Field(description="New participants to add")
    participants_to_remove: list[str] = Field(description="Participants to remove")


# --------------------------------------------------------------
# Step 2: Define the routing and processing functions
# --------------------------------------------------------------

In [ ]:
def route_calendar_request(user_input:str) -> CalendarRequestType:
    messages = [
        {"role": "system","content": "Determine if this is a request to create a new calendar event or modify an existing one.",},
        {"role": "user", "content": user_input},
        ]

    response = client.chat.completions.parse(
        model=MODEL_NAME,
        temperature=0.7,
        n=1,
        messages=messages,
        response_format=CalendarRequestType
        )
    
    print(f"Response from {MODEL_NAME} on {API_HOST}")
    #print(response)
    parsed_CalendarRequestType = response.choices[0].message.parsed
    print("parsed_CalendarRequestType:",parsed_CalendarRequestType)
    
    return parsed_CalendarRequestType


def handle_new_event(desc) -> CalendarResponse:
    messages = [
        {"role": "system","content": "Extract details for creating a new calendar event.",},
        {"role": "user", "content": desc},
        ]

    response = client.chat.completions.parse(
        model=MODEL_NAME,
        temperature=0.7,
        n=1,
        messages=messages,
        response_format=NewEventDetails
        )
    
    print(f"Response from {MODEL_NAME} on {API_HOST}")
    parsed_NewEventDetails = response.choices[0].message.parsed
    print("parsed_NewEventDetails:",parsed_NewEventDetails)

    return CalendarResponse(
        success=True,
        message=f"Created new event '{parsed_NewEventDetails.name}' for {parsed_NewEventDetails.date} with {', '.join(parsed_NewEventDetails.participants)}",
        calendar_link=f"calendar://new?event={parsed_NewEventDetails.name}",
    )

def handle_modify_event(desc)->CalendarResponse:
    messages=[
            {
                "role": "system",
                "content": "Extract details for modifying an existing calendar event.",
            },
            {"role": "user", "content": desc},
        ]



    response = client.chat.completions.parse(
        model=MODEL_NAME,
        temperature=0.7,
        n=1,
        messages=messages,
        response_format=ModifyEventDetails,

        )
    
    print(f"Response from {MODEL_NAME} on {API_HOST}")
    parsed_ModifyEventDetails = response.choices[0].message.parsed
    print("parsed_ModifyEventDetails:",parsed_ModifyEventDetails)

    # here you can call the Calendar API to make the actual changes



# --------------------------------------------------------------
# ############# MAIN CODE ####################
# --------------------------------------------------------------

In [27]:
# test for new event
#user_input = "Let's schedule a team meeting next Tuesday at 2pm with Alice and Bob"

# test for modify event
#user_input = "Can you move the team meeting with Alice and Bob to Wednesday at 3pm instead?"

## test for invalid event
user_input = "What's the weather like today"
route_result = route_calendar_request(user_input)

Response from gpt-4o-mini on github

parsed_CalendarRequestType:
CalendarRequestType(
    request_type='other',
    confidence_score=0.0,
    description='The request is not related to calendar events.'
)

In [28]:
# Check confidence threshold
if route_result.confidence_score < 0.7:
    print("Exiting program due to low confidence score...")
    exit(1)

# Now based on the request_type, we need to route to the right LLM agent

# Lets first check the event type
route_result = route_calendar_request(user_input)

if route_result.request_type == 'new_event':
    response = handle_new_event(route_result.description)
elif route_result.request_type == "modify_event":
    response = handle_modify_event(route_result.description)

print(response)

Exiting program due to low confidence score...

Response from gpt-4o-mini on github

parsed_CalendarRequestType:
CalendarRequestType(
    request_type='other',
    confidence_score=0.0,
    description='The request does not pertain to a calendar event.'
)

None